## Import packages

In [1]:
# Standard libraries
import requests
import pandas as pd
import numpy as np

## Pull fixtures and team data from the API 

In [33]:
# FPL API for fixtures
url = 'https://fantasy.premierleague.com/api/fixtures/'
response = requests.get(url)
fixtures_json = response.json()

# store in pandas DF
fixtures_df = pd.DataFrame(fixtures_json)
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2293083,NaN,False,False,274,None,0,False,None,14,NaN,5,NaN,[],3,3,75184
1,2293085,NaN,False,False,276,None,0,False,None,9,NaN,12,NaN,[],2,4,75186
2,2293086,NaN,False,False,277,None,0,False,None,19,NaN,13,NaN,[],2,5,75187
3,2293057,NaN,False,False,248,None,0,False,None,5,NaN,15,NaN,[],3,4,75158
4,2292810,1.0,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74911


In [34]:
# FPL API for teams
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
response = requests.get(url)
json = response.json()

# storing json outputs as dataframes
teams_df = pd.DataFrame(json['teams'])
teams_df.head()

,code,draw,form,id,loss,name,played,points,position,short_name,...,team_division,unavailable,win,strength_overall_home,strength_overall_away,strength_attack_home,strength_attack_away,strength_defence_home,strength_defence_away,pulse_id
0,3,0,None,1,0,Arsenal,0,0,0,ARS,...,None,False,0,1245,1285,1250,1250,1240,1320,1
1,7,0,None,2,0,Aston Villa,0,0,0,AVL,...,None,False,0,1070,1100,1070,1075,1070,1130,2
2,91,0,None,3,0,Bournemouth,0,0,0,BOU,...,None,False,0,1035,1095,1020,1110,1050,1080,127
3,94,0,None,4,0,Brentford,0,0,0,BRE,...,None,False,0,1115,1180,1100,1160,1130,1200,130
4,36,0,None,5,0,Brighton,0,0,0,BHA,...,None,False,0,1170,1175,1140,1150,1200,1200,131


## Join the tables to get team names into the fixtures table

In [35]:
# getting team names into fixtures table
fixtures_df['home_team'] = pd.merge(fixtures_df, teams_df, left_on='team_h', right_on='id', how='left')['name']
fixtures_df['away_team'] = pd.merge(fixtures_df, teams_df, left_on='team_a', right_on='id', how='left')['name']

fixtures_df[fixtures_df['finished'] == True].head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,home_team,away_team
4,2292810,1.0,True,True,1,2022-08-05T19:00:00Z,90,False,True,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74911,Crystal Palace,Arsenal
5,2292813,1.0,True,True,4,2022-08-06T11:30:00Z,90,False,True,12,2.0,9,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,74914,Fulham,Liverpool
6,2292811,1.0,True,True,2,2022-08-06T14:00:00Z,90,False,True,2,0.0,3,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,2,74912,Bournemouth,Aston Villa
7,2292814,1.0,True,True,5,2022-08-06T14:00:00Z,90,False,True,20,1.0,11,2.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",2,2,74915,Leeds,Wolves
8,2292816,1.0,True,True,7,2022-08-06T14:00:00Z,90,False,True,16,0.0,15,2.0,"[{'identifier': 'goals_scored', 'a': [], 'h': ...",2,4,74917,Newcastle,Nott'm Forest


## Retain only necessary columns

In [39]:
# removing unnecessary columns
fixtures_df = fixtures_df[['id', 'event', 'finished', 'kickoff_time', 'team_a', 'team_a_score', 'team_h', 'team_h_score', 'stats',
                           'team_h_difficulty', 'team_a_difficulty', 'home_team', 'away_team']]

fixtures_df.head()

,id,event,finished,kickoff_time,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,home_team,away_team
0,274,NaN,False,None,14,NaN,5,NaN,[],3,3,Brighton,Man Utd
1,276,NaN,False,None,9,NaN,12,NaN,[],2,4,Liverpool,Fulham
2,277,NaN,False,None,19,NaN,13,NaN,[],2,5,Man City,West Ham
3,248,NaN,False,None,5,NaN,15,NaN,[],3,4,Newcastle,Brighton
4,1,1.0,True,2022-08-05T19:00:00Z,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,Crystal Palace,Arsenal


In [43]:
fixtures_df = fixtures_df.rename(columns={'id': 'match_id', 'event': 'gameweek', 'team_a': 'away_team_id', 'team_h': 'home_team_id'})
fixtures_df.head()

,match_id,gameweek,finished,kickoff_time,away_team_id,team_a_score,home_team_id,team_h_score,stats,team_h_difficulty,team_a_difficulty,home_team,away_team
0,274,NaN,False,None,14,NaN,5,NaN,[],3,3,Brighton,Man Utd
1,276,NaN,False,None,9,NaN,12,NaN,[],2,4,Liverpool,Fulham
2,277,NaN,False,None,19,NaN,13,NaN,[],2,5,Man City,West Ham
3,248,NaN,False,None,5,NaN,15,NaN,[],3,4,Newcastle,Brighton
4,1,1.0,True,2022-08-05T19:00:00Z,1,2.0,7,0.0,"[{'identifier': 'goals_scored', 'a': [{'value'...",4,2,Crystal Palace,Arsenal
